In [4]:
import numpy as np
import pandas as pd

import helpers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [5]:
def add_installments_payment_features(df, ip_df):
    features = []

    # statistics
    new_cols = []
    installments_payment_numerical_stats = ip_df.drop(columns=['SK_ID_PREV']).groupby('SK_ID_CURR', as_index = False).agg(['median', 'mean', 'max', 'min', 'sum']).reset_index()
    for feature in installments_payment_numerical_stats.columns.levels[0]:
        if feature == 'SK_ID_CURR':
            continue
        for stat in installments_payment_numerical_stats.columns.levels[1][:-1]:
            new_cols.append('installments_payment___{}_{}'.format(feature, stat))
    installments_payment_numerical_stats.columns = ['SK_ID_CURR'] + new_cols
    features += new_cols
    df = df.merge(installments_payment_numerical_stats, on='SK_ID_CURR', how='left')

    #days_payment_diff
    features.append('installments_payment__days_payment_diff')
    ip_df['installments_payment__days_payment_diff'] = ip_df.DAYS_ENTRY_PAYMENT - ip_df.DAYS_INSTALMENT
    days_payment_diff = ip_df.groupby('SK_ID_PREV')[['SK_ID_CURR', 'installments_payment__days_payment_diff']].mean().groupby('SK_ID_CURR').mean()
    df = df.merge(days_payment_diff, on='SK_ID_CURR', how='left')
    df['installments_payment__days_payment_diff'].fillna(0, inplace=True)
    #amt_payment_diff
    features.append('installments_payment__amt_payment_diff')
    ip_df['installments_payment__amt_payment_diff'] = ip_df.AMT_INSTALMENT - ip_df.AMT_PAYMENT
    amt_payment_diff = ip_df.groupby('SK_ID_PREV')[['SK_ID_CURR', 'installments_payment__amt_payment_diff']].mean().groupby('SK_ID_CURR').mean()
    df = df.merge(amt_payment_diff, on='SK_ID_CURR', how='left')
    df['installments_payment__amt_payment_diff'].fillna(0, inplace=True)

    return df, features

In [6]:
application = pd.read_csv('data/application_test.csv')
installments_payment = pd.read_csv('data/installments_payments.csv')

In [3]:
installments_payment.shape

(13605401, 8)

In [11]:
installments_payment.sample(5)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,installments_payment__days_payment_diff,installments_payment__amt_payment_diff
6650770,2761322,220192,1.0,4,-352.0,-360.0,26027.865,26027.865,-8.0,0.0
498003,1568915,121359,1.0,7,-253.0,-262.0,41966.505,41966.505,-9.0,0.0
11502168,1082323,417051,1.0,30,-1602.0,-1611.0,8478.000,8478.000,-9.0,0.0
9589511,2660598,379133,0.0,66,-241.0,-241.0,7237.530,7237.530,0.0,0.0
8154829,2161613,304027,0.0,24,-231.0,-250.0,275.850,275.850,-19.0,0.0


In [12]:
np.unique(installments_payment.NUM_INSTALMENT_VERSION)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  61.,  68.,  72.,  73., 178.])

In [7]:
application, installments_payment_features = add_installments_payment_features(application, installments_payment)

In [8]:
application[installments_payment_features].sample(5)

,installments_payment___NUM_INSTALMENT_VERSION_median,installments_payment___NUM_INSTALMENT_VERSION_mean,installments_payment___NUM_INSTALMENT_VERSION_max,installments_payment___NUM_INSTALMENT_VERSION_min,installments_payment___NUM_INSTALMENT_VERSION_sum,installments_payment___NUM_INSTALMENT_NUMBER_median,installments_payment___NUM_INSTALMENT_NUMBER_mean,installments_payment___NUM_INSTALMENT_NUMBER_max,installments_payment___NUM_INSTALMENT_NUMBER_min,installments_payment___NUM_INSTALMENT_NUMBER_sum,installments_payment___DAYS_INSTALMENT_median,installments_payment___DAYS_INSTALMENT_mean,installments_payment___DAYS_INSTALMENT_max,installments_payment___DAYS_INSTALMENT_min,installments_payment___DAYS_INSTALMENT_sum,installments_payment___DAYS_ENTRY_PAYMENT_median,installments_payment___DAYS_ENTRY_PAYMENT_mean,installments_payment___DAYS_ENTRY_PAYMENT_max,installments_payment___DAYS_ENTRY_PAYMENT_min,installments_payment___DAYS_ENTRY_PAYMENT_sum,installments_payment___AMT_INSTALMENT_median,installments_payment___AMT_INSTALMENT_mean,installments_payment___AMT_INSTALMENT_max,installments_payment___AMT_INSTALMENT_min,installments_payment___AMT_INSTALMENT_sum,installments_payment___AMT_PAYMENT_median,installments_payment___AMT_PAYMENT_mean,installments_payment___AMT_PAYMENT_max,installments_payment___AMT_PAYMENT_min,installments_payment___AMT_PAYMENT_sum,installments_payment__days_payment_diff,installments_payment__amt_payment_diff
41465,1.0,1.333333,2.0,1.0,12.0,3.0,3.333333,7.0,1.0,30.0,-536.0,-537.111111,-417.0,-656.0,-4834.0,-544.0,-544.444444,-417.0,-667.0,-4900.0,18282.555,187864.090000,765521.865,18282.555,1690776.810,18282.555,187864.090000,765521.865,18282.555,1690776.810,-3.142857,0.000000
32782,1.0,1.285714,3.0,1.0,198.0,10.0,11.616883,31.0,1.0,1789.0,-750.0,-901.961039,-30.0,-2573.0,-138902.0,-775.0,-911.603896,-42.0,-2581.0,-140387.0,11071.530,11229.854610,47765.835,2366.730,1729397.610,11071.530,10909.093734,47765.835,22.950,1680000.435,-11.829514,106.399500
1556,0.0,0.055556,1.0,0.0,6.0,55.5,55.444444,109.0,1.0,5988.0,-1457.0,-1502.907407,-10.0,-2903.0,-162314.0,-1477.5,-1513.972222,-10.0,-2920.0,-163509.0,9000.000,11278.068750,15990.075,67.500,1218031.425,9000.000,11278.068750,15990.075,67.500,1218031.425,-9.936275,0.000000
9233,1.0,1.000000,1.0,1.0,26.0,4.5,5.269231,12.0,1.0,137.0,-520.0,-414.307692,-31.0,-835.0,-10772.0,-528.5,-425.269231,-41.0,-840.0,-11057.0,8418.690,8933.654423,10596.420,7413.120,232275.015,8418.690,8286.062885,10596.420,581.310,215437.635,-11.363095,400.890000
34844,1.0,1.000000,1.0,1.0,40.0,5.0,5.050000,10.0,1.0,202.0,-950.0,-1078.675000,-606.0,-1786.0,-43147.0,-951.0,-1087.300000,-596.0,-1805.0,-43492.0,7114.635,8189.791875,11250.225,5189.805,327591.675,6013.350,6090.854625,11250.225,123.570,243634.185,-8.842692,1276.751791


In [10]:
installments_payment.groupby('SK_ID_PREV')[['SK_ID_CURR', 'installments_payment__days_payment_diff']].mean().shape

(997752, 2)